# In this note book we're going to cover some of the most fundamental concepts of tensors using TensorFlow
More specifically, we're going to cover:
* introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try myself

## Introduction to Tensors

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

In [ ]:
# Check number of dims of a tensor (ndim = number of dimensions)
scalar.ndim

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

In [ ]:
# Dim of vector
vector.ndim

In [ ]:
# create a matrix
matrix = tf.constant([[10, 7], [7, 10]])
matrix

In [ ]:
# Dim of matrix
matrix.ndim

In [ ]:
# Another matrix
another_matrix = tf.constant([[10., 7.], 
    [3., 2.], 
    [8., 9.]], 
    dtype=tf.float16)
another_matrix

In [ ]:
# Dim of another_matrix
another_matrix.ndim

In [ ]:
# Create a tensor
tensor = tf.constant(
      [[[1, 2, 3],
        [4, 5, 6]],
       [[7, 8, 9],
        [10, 11, 12]],
       [[13, 14, 15],
        [16, 17, 18]]])
tensor

In [ ]:
tensor.ndim

So far:
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-d array of numbers
* Tensor: a n-d array of numbers 

### Creating tensors with tf.variable

In [ ]:
# Create the same tensors with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

In [ ]:
# Try to change elements
changeable_tensor[0] = 7
changeable_tensor

In [ ]:
# try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# try at unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

***
🔑 **Note:** Rarely in practive i will need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for me. However, if in doubt, i'm going to use `tf.constant` and change it later if needed.
***

### Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers

In [31]:
# Create to random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)